In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
!pip install spacy nltk transformers
import spacy

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.0 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/10.0 MB 4.2 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/10.0 MB 4.4 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/10.0 MB 4.4 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/10.0 MB 3.0 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/10.0 MB 2.8 MB/s eta 0:00:03
   ---------------- ----------------------- 4.2/10.0 MB 2.9 MB/s eta 0:00:03
   ------------------- -------------------- 5.0/10.0 MB 3.0 MB/s eta 0:00:02
   -------------------- ------------------- 5.2/10.0 MB 2.8 MB/s eta 0:00:02
   ------------------------- -------------- 6.3/10.0 MB 3.0 MB/s eta 0:00:02
   -------------------------- ------------- 6.6/10.0 MB 2.9 MB/s eta 0:00:02
   ----------

In [6]:
#loading dataset
data = pd.read_csv("eng-french.csv", nrows = 50)
data.columns = ['english', 'french']

# Clean 'english' column
data['english'] = data['english'].str.lower()
data['english'] = data['english'].apply(lambda x: re.sub(r"[^a-z\s]", "", x))
data['english'] = data['english'].apply(lambda x: re.sub(r"\s+", " ", x).strip())
# Clean 'french' column
data['french'] = data['french'].str.lower()
data['french'] = data['french'].apply(lambda x: re.sub(r"[^a-z\s]", "", x))
data['french'] = data['french'].apply(lambda x: re.sub(r"\s+", " ", x).strip())

data.head()

,english,french
0,hi,salut
1,run,cours
2,run,courez
3,who,qui
4,wow,a alors


In [7]:
#sub word tokenization
from transformers import AutoTokenizer

# Load a pre-trained tokenizer (e.g., BERT tokenizer)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize English and French columns
data['english_subtokens'] = data['english'].apply(lambda x: tokenizer.tokenize(x))
data['french_subtokens'] = data['french'].apply(lambda x: tokenizer.tokenize(x))

# View tokenized data
print(data[['english', 'english_subtokens']].head())


c:\Users\Siraj\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Siraj\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Siraj\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adminis

  english english_subtokens
0      hi              [hi]
1     run             [run]
2     run             [run]
3     who             [who]
4     wow             [wow]


In [12]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 2.4 MB/s eta 0:00:06
     ----- ---------------------------------- 1.8/12.8 MB 3.9 MB/s eta 0:00:03
     ------- -------------------------------- 2.4/12.8 MB 4.2 MB/s eta 0:00:03
     -------- ------------------------------- 2.6/12.8 MB 3.1 MB/s eta 0:00:04
     ----------- ---------------------------- 3.7/12.8 MB 3.5 MB/s eta 0:00:03
     ------------- -------------------------- 4.5/12.8 MB 3.5 MB/s eta 0:00:03
     ------------- -------------------------- 4.5/12.8 MB 3.5 MB/s eta 0:00:03
     ---------------- ----------------------- 5.2/12.8 MB 3.0 MB/s eta 0:00:03
     ------------------ --------------------- 6.0/12.8 MB 3.1 MB/s eta 0:00:03
     ------------------- -------------------- 6.3/12.8 MB 3.1 MB/s eta 0:

In [13]:
#parsing into grammar Trees
nlp = spacy.load("en_core_web_sm")
# Extract dependency relationships (grammar trees)
data['english_grammar'] = data['english'].apply(lambda x: [(token.text, token.dep_, token.head.text) for token in nlp(x)])
data['french_grammar'] = data['french'].apply(lambda x: [(token.text, token.dep_, token.head.text) for token in nlp(x)])

# View grammar structures
print(data[['english', 'english_grammar']].head())


  english     english_grammar
0      hi    [(hi, ROOT, hi)]
1     run  [(run, ROOT, run)]
2     run  [(run, ROOT, run)]
3     who  [(who, ROOT, who)]
4     wow  [(wow, ROOT, wow)]
